### Property Lookup Resolution

Как мы увидели в последнем наборе лекций, происходит что-то странное, когда наш класс использует дескриптор данных, а экземпляры содержат одно и то же имя атрибута в словаре экземпляров.

Вопреки нашим ожиданиям, дескриптор **все еще** использовался.

Это сводится к дескрипторам данных и не-данных. У Python есть способ по умолчанию, где он ищет атрибуты в зависимости от того, является ли дескриптор дескриптором данных или нет.

Для дескрипторов данных Python выберет использование атрибута дескриптора (в классе), даже если тот же символ найден в словаре экземпляров.

Давайте рассмотрим это еще раз на простом примере:

In [1]:
class IntegerValue:
    def __set__(self, instance, value):
        print('__set__ called...')

    def __get__(self, instance, owner_class):
        print('__get__ called...')

In [2]:
class Point:
    x = IntegerValue()

In [3]:
p = Point()

In [4]:
p.x = 100

__set__ called...


In [5]:
p.x

__get__ called...


Итак, были вызваны методы дескриптора `__set__` и `__get__`.

Давайте установим атрибут с именем `x` непосредственно в словаре экземпляра:

In [6]:
p.__dict__

{}

In [7]:
p.__dict__['x'] = 'hello'

In [8]:
p.__dict__

{'x': 'hello'}

А теперь давайте получим значение:

In [9]:
p.x

__get__ called...


Как вы видите, дескриптор **все еще** использовался. То же самое, если мы установим значение:

In [10]:
p.x = 100

__set__ called...


Это работает таким образом, потому что у нас есть **дескриптор данных** — атрибуты экземпляра не затеняют дескрипторы классов с тем же именем!

Поведение для дескриптора, не являющегося данными, отличается, и эффект затенения присутствует:

In [11]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        print('__get__ called...')
        return datetime.utcnow().isoformat()

In [12]:
class Logger:
    current_time = TimeUTC()

In [13]:
l = Logger()

In [14]:
l.current_time

__get__ called...


'2019-07-13T20:47:59.473945'

Как вы можете видеть, был вызван `__get__` дескриптора.

Теперь давайте внедрим тот же символ непосредственно в наш словарь экземпляров:

In [15]:
l.__dict__

{}

In [16]:
l.__dict__['current_time'] = 'this is not a timestamp'

In [17]:
l.__dict__

{'current_time': 'this is not a timestamp'}

И если мы попытаемся получить значение для этого ключа:

In [18]:
l.current_time

'this is not a timestamp'

мы получаем значение, хранящееся в словаре экземпляра, а **не** метод дескриптора `__get__`.

Конечно, мы можем вернуться к «нормальному» состоянию, удалив этот ключ из словаря экземпляра:

In [19]:
del l.__dict__['current_time']

А теперь:

In [20]:
l.current_time

__get__ called...


'2019-07-13T20:47:59.556109'

Это означает, что для дескрипторов данных, где нам обычно требуется хранилище на основе экземпляра, мы можем фактически использовать само имя свойства для сохранения значения в экземпляре **под тем же именем**. Он **не** будет затенять атрибут класса (экземпляр дескриптора), и у него нет риска перезаписать какие-либо существующие атрибуты экземпляра, которые может иметь наш класс!

Конечно, это предполагает, что класс не использует слоты или, по крайней мере, указывает `__dict__` в качестве одного из слотов, если он это делает.

Давайте применим это к дескриптору данных при таком предположении:

In [21]:
class ValidString:
    def __init__(self, min_length):
        self.min_length = min_length

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.prop_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.prop_name} must be '
                             f'at least {self.min_length} characters.'
                            )
        instance.__dict__[self.prop_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [22]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [23]:
p = Person()

In [24]:
p.__dict__

{}

In [25]:
p.first_name = 'Alex'
p.last_name = 'Martelli'

In [26]:
p.__dict__

{'first_name': 'Alex', 'last_name': 'Martelli'}

In [27]:
p.first_name, p.last_name

('Alex', 'Martelli')

Обратите внимание, что я **не** использую атрибуты (ни точечную нотацию, ни `getattr`/`setattr`) при установке и получении значений из экземпляра `__dict__`. Если бы я это сделал, это фактически вызвало бы методы дескрипторов `__get__` и `__set__`, что привело бы к бесконечной рекурсии!!

Так что будьте осторожны с этим!

In [28]:
class ValidString:
    def __init__(self, min_length):
        self.min_length = min_length

    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name

    def __set__(self, instance, value):
        print('calling __set__ ...')
        if not isinstance(value, str):
            raise ValueError(f'{self.prop_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.prop_name} must be '
                             f'at least {self.min_length} characters.'
                            )
        setattr(instance, self.prop_name, value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [29]:
class Person:
    name = ValidString(1)

In [30]:
p = Person()

In [31]:
p.name = 'Alex'

calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...


calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...
calling __set__ ...


RecursionError: maximum recursion depth exceeded in comparison